In [1]:
import pandas as pd

In [2]:
jsonFile = pd.read_json("../Traza.json")

In [3]:
x = pd.json_normalize(jsonFile['data'])
groupedEvents = x.groupby('eventId')
for eid, group in groupedEvents:
    display(group)
    ##display(eid)

,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
0,0,1713868496125,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,SteamMazehemGame,NaN,NaN,NaN


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
24,1,1713868600442,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
1,4,1713868499980,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,0.0,NaN,NaN
22,4,1713868583724,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,1.0,NaN,NaN


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
21,5,1713868583724,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,0.0,NaN,0.0


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
9,6,1713868539108,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
23,6,1713868598570,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
11,7,1713868553721,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
4,8,1713868535754,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
7,8,1713868536843,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
10,8,1713868539713,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
16,8,1713868567031,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
18,8,1713868568755,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
5,9,1713868535755,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
8,9,1713868536843,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
17,9,1713868567031,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
19,9,1713868568755,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
2,10,1713868523991,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,2.0,NaN
3,10,1713868535711,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,0.0,NaN
6,10,1713868536720,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,0.0,NaN
15,10,1713868566504,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,1.0,NaN
20,10,1713868581761,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,2.0,NaN


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
12,12,1713868557251,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
13,12,1713868560953,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
14,12,1713868561553,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN


### Los eventos y sus ID:
- SessionStart = 0
- SessionStop = 1
- GameStart = 2
- GameEnd = 3
- LevelStart = 4
- LevelEnd = 5
- Pause = 6
- Resume = 7
- //
- Attack = 8
- EnemyReceive = 9
- PlayerReceive = 10
- PlayerDead = 11
- RoomMove = 12


# Métrica 1: Tiempo de finalización de nivel
¿El jugador tarda demasiado tiempo en completar cada uno de los dos niveles?

In [4]:
from datetime import datetime

## estoy usando los de la sesion por ahora, pero se haria con los de cada nivel.
startEvent = groupedEvents.get_group(0)
display(startEvent)
endEvent = groupedEvents.get_group(1)
display(endEvent)

a = startEvent.get("timestamp").values[0]
b = endEvent.get("timestamp").values[0]

# Convertir el timestamp de milisegundos a segundos
a_seconds = a / 1000
b_seconds = b / 1000
total_pause_time = 0
# ver si hay pausas
if 6 in groupedEvents.groups and 7 in groupedEvents.groups:
    # Si existen, obtén los eventos de pausa y reanudación
    pauseEvents = groupedEvents.get_group(6)
    resumeEvents = groupedEvents.get_group(7)

    # Asegúrate de que los eventos de pausa y reanudación estén ordenados por timestamp
    pauseEvents = pauseEvents.sort_values("timestamp")
    resumeEvents = resumeEvents.sort_values("timestamp")

    # Recorre los eventos de pausa y reanudación
    for i in range(len(resumeEvents)):
        pause_time = pauseEvents.iloc[i]["timestamp"]
        resume_time = resumeEvents.iloc[i]["timestamp"]

        # Añade la duración de esta pausa al tiempo total de pausa
        total_pause_time += resume_time - pause_time

# Convierte el tiempo total de pausa de milisegundos a segundos
total_pause_time_seconds = total_pause_time / 1000

# Ahora puedes restar el tiempo total de pausa del tiempo total de la sesión
session_duration = (b_seconds - a_seconds) - total_pause_time_seconds

print("Duración de la sesión con pausas descontadas: ", session_duration/60, " minutos")

print("TIME: ")
print((b_seconds - a_seconds)/60, " minutos")

# Crear objetos datetime a partir de los timestamps
dt_object_start = datetime.fromtimestamp(a_seconds)
dt_object_end = datetime.fromtimestamp(b_seconds)

# Imprimir los objetos datetime y sus tipos
print("Start time: ", dt_object_start)
print("End time: ", dt_object_end)

,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
0,0,1713868496125,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,SteamMazehemGame,NaN,NaN,NaN


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
24,1,1713868600442,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN


Duración de la sesión con pausas descontadas:  1.495066665204366  minutos
TIME: 
1.738616665204366  minutos
Start time:  2024-04-23 12:34:56.125000
End time:  2024-04-23 12:36:40.442000


# Métrica 2: Cantidad de movimientos por nivel
¿El jugador usa demasiados movimientos de salas en cada uno de los dos niveles?

In [5]:
# FALTA AGRUPAR POR NIVELES
room_moves = groupedEvents.get_group(12)
display(room_moves)

# Se agrupan los movimientos por sesión y se cuenta el número de cada una
movements = room_moves.groupby('Session').size().reset_index(name='Movements')

print("Session movements:\n")
print(movements.to_string(index=False))

,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
12,12,1713868557251,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
13,12,1713868560953,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
14,12,1713868561553,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN


Session movements:

                             Session  Movements
eb92b00f-ba3f-4f4a-b0df-81e6864c9fec          3


# Métrica 3: Precisión de golpes a los enemigos
¿El jugador falla significativamente más ataques en el nivel 2 que en el nivel 1?

In [6]:
# FALTA AGRUPAR POR NIVELES
# Verificar si los grupos existen antes de acceder a ellos
if 8 in groupedEvents.groups:
    attack = groupedEvents.get_group(8)
    enemy_receive = groupedEvents.get_group(9) if 9 in groupedEvents.groups else None
    
    display(attack)
    display(enemy_receive)

    # Se agrupan los ataques realizados y los recibidos por los enemigos y se cuenta el número de cada uno
    attack_count = attack.groupby('Session').size().reset_index(name='Total Attacks')
    
    # Si el grupo 9 existe, contamos los aciertos; de lo contrario, asumimos que son 0
    if enemy_receive is not None:
        hit_count = enemy_receive.groupby('Session').size().reset_index(name='Total Hits')
    else:
        hit_count = pd.DataFrame({'Session': attack_count['Session'], 'Total Hits': 0})
    
    # Agrupar los datos en un único dataframe
    accuracy = pd.merge(attack_count, hit_count, on='Session', how='left')
    accuracy['Total Hits'] = accuracy['Total Hits'].fillna(0)

    # Calcular la precisión de los golpes
    accuracy['Accuracy'] = (accuracy['Total Hits'] / accuracy['Total Attacks']) * 100
    accuracy['Accuracy'] = accuracy['Accuracy'].apply(lambda x: f"{x:.3f}%")

    print("Hit accuracy:\n")
    print(accuracy[['Session', 'Accuracy']].to_string(index=False))
else:
    print("El grupo 8 no existe en el DataFrame 'groupedEvents'. No se pueden calcular las estadísticas.")


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
4,8,1713868535754,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
7,8,1713868536843,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
10,8,1713868539713,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
16,8,1713868567031,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
18,8,1713868568755,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN


,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
5,9,1713868535755,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
8,9,1713868536843,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
17,9,1713868567031,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN
19,9,1713868568755,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,NaN,NaN


Hit accuracy:

                             Session Accuracy
eb92b00f-ba3f-4f4a-b0df-81e6864c9fec  80.000%


# Métrica 4: Impactos de los elementos dañinos
¿Es coherente la variación del daño provocado por cada elemento dañino a lo largo de los dos niveles?


In [7]:
# FALTA AGRUPAR POR NIVELES
player_receive = groupedEvents.get_group(10)
display(player_receive)

# Se agrupan los impactos por sesión y tipo de enemigo y se cuenta el número de cada una
impact_count = player_receive.groupby(['Session', 'EnemyType']).size().reset_index(name='Impacts')

# Impactos totales por sesión
total_impact_count = player_receive.groupby('Session').size().reset_index(name='Total Impacts')

# Agrupar los datos en un único dataframe
impact_count = impact_count.merge(total_impact_count, on='Session', how='left')

# Calcular el porcentaje de impactos provocados por cada tipo de enemigo
impact_count['Enemy Percentage'] = (impact_count['Impacts'] / impact_count['Total Impacts']) * 100
impact_count['Enemy Percentage'].fillna(0, inplace=True)
impact_count['Enemy Percentage'] = impact_count['Enemy Percentage'].apply(lambda x: f"{x:.3f}%")

print("Enemy impacts:\n")
print(impact_count[['Session', 'EnemyType', 'Impacts', 'Enemy Percentage']].to_string(index=False))
print("\nTotal impacts: ", impact_count['Impacts'].sum())

,eventId,timestamp,Session,User,Game,LevelID,EnemyType,LevelEnd
2,10,1713868523991,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,2.0,NaN
3,10,1713868535711,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,0.0,NaN
6,10,1713868536720,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,0.0,NaN
15,10,1713868566504,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,1.0,NaN
20,10,1713868581761,eb92b00f-ba3f-4f4a-b0df-81e6864c9fec,54f930cf-a627-422c-9f29-4b2271a347c9,NaN,NaN,2.0,NaN


Enemy impacts:

                             Session  EnemyType  Impacts Enemy Percentage
eb92b00f-ba3f-4f4a-b0df-81e6864c9fec        0.0        2          40.000%
eb92b00f-ba3f-4f4a-b0df-81e6864c9fec        1.0        1          20.000%
eb92b00f-ba3f-4f4a-b0df-81e6864c9fec        2.0        2          40.000%

Total impacts:  5
